In [ ]:
#!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to c:\users\14122\appdata\local\temp\pip-req-build-no9bx28e
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369594 sha256=fc50eaa1cf700f0524a5c865eacfe20700710195f33d03f80068317007a0356b
  Stored in directory: C:\Users\14122\AppData\Local\Temp\pip-ephem-wheel-cache-xpxmpw8p\wheels\da\2b\4c\d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\14122\AppData\Local\Temp\pip-req-build-no9bx28e'


## 1. Data Preparation

In [16]:
image_path_MetMeme = "C:/Users/14122/Desktop/test data/Met-Meme/CLIP_Basic/Image"


### Image Preprocessing

In [2]:

from PIL import Image
import clip
model, preprocess = clip.load("ViT-B/32", device="cuda")

def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    return preprocess(image)  # 返回形状 [3, 224, 224]

100%|███████████████████████████████████████| 338M/338M [00:05<00:00, 68.8MiB/s]


In [33]:
image = load_image("C:/Users/14122/Desktop/test data/Met-Meme/CLIP_Basic/Image/image_ (1).jpg")
image

tensor([[[0.9522, 0.9522, 0.9522,  ..., 0.9522, 0.9522, 0.9522],
         [0.9522, 0.9522, 0.9522,  ..., 0.9522, 0.9522, 0.9522],
         [0.9522, 0.9522, 0.9522,  ..., 0.9522, 0.9522, 0.9522],
         ...,
         [0.9522, 0.9522, 0.9522,  ..., 0.9522, 0.9522, 0.9522],
         [0.9522, 0.9522, 0.9522,  ..., 0.9522, 0.9522, 0.9522],
         [0.9522, 0.9522, 0.9522,  ..., 0.9522, 0.9522, 0.9522]],

        [[1.3995, 1.3995, 1.3995,  ..., 1.3995, 1.3995, 1.3995],
         [1.3995, 1.3995, 1.3995,  ..., 1.3995, 1.3995, 1.3995],
         [1.3995, 1.3995, 1.3995,  ..., 1.3995, 1.3995, 1.3995],
         ...,
         [1.3995, 1.3995, 1.3995,  ..., 1.3995, 1.3995, 1.3995],
         [1.3995, 1.3995, 1.3995,  ..., 1.3995, 1.3995, 1.3995],
         [1.3995, 1.3995, 1.3995,  ..., 1.3995, 1.3995, 1.3995]],

        [[0.8377, 0.8377, 0.8377,  ..., 0.8377, 0.8377, 0.8377],
         [0.8377, 0.8377, 0.8377,  ..., 0.8377, 0.8377, 0.8377],
         [0.8377, 0.8377, 0.8377,  ..., 0.8377, 0.8377, 0.

### Text Preprocessing

In [34]:
# 修改process_text以适配数据集：
def process_text(text):
    return clip.tokenize([text])  # 不再添加前缀，更灵活


In [35]:
# 在数据集中：
text = process_text(row["Overall Tone"])

NameError: name 'row' is not defined

### Label Encode

In [5]:
#!pip install scikit-learn

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 11.1/11.1 MB 86.9 MB/s eta 0:00:00


In [12]:
import sklearn
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the CSV file
df = pd.read_csv("C:/Users/14122/Desktop/test data/Met-Meme/CLIP_Basic/modified_data.csv")

# Select the "intention_detection" column
intention_column = df["intention detection"]

# Encode the labels
#labels = encoder.fit_transform(intention_column)  # Encodes values like 1, 2, 3, 4, 5 into 0, 1, 2, 3, 4

encoder = LabelEncoder()
labels = encoder.fit_transform(intention_column)  # 输出 [0, 1, 2, 3, 4]

In [13]:
# Check the mapping of original labels to encoded labels
label_mapping = dict(zip(encoder.classes_, range(len(encoder.classes_))))
print("Label Mapping:", label_mapping)

# Verify the first few encoded labels
print("Encoded Labels (first 10):", labels[:10])

Label Mapping: {'1(interactive)': 0, '2(expressive)': 1, '3(entertaining)': 2, '4(offensive)': 3, '5(other)': 4}
Encoded Labels (first 10): [3 1 1 3 2 2 2 2 2 2]


In [21]:
df["label_index"] = labels  # Add the labels to DataFrame 中
# Check the mapping of original labels to encoded labels
label_mapping = dict(zip(encoder.classes_, range(len(encoder.classes_))))
print("Label Mapping:", label_mapping)

# Verify the first few rows of the DataFrame
print(df.head())

Label Mapping: {'1(interactive)': 0, '2(expressive)': 1, '3(entertaining)': 2, '4(offensive)': 3, '5(other)': 4}
          image_file                                       Overall Tone  \
0     image_ (0).jpg   The tone is lighthearted and self-deprecating...   
1     image_ (1).jpg   Playful and lighthearted. The image uses humo...   
2    image_ (10).jpg  \n- The image conveys a sense of irony and hum...   
3   image_ (100).jpg  \n- The meme uses humor and exaggeration to po...   
4  image_ (1000).jpg   The tone of the image is contemplative and ph...   

  intention detection  label_index  
0        4(offensive)            3  
1       2(expressive)            1  
2       2(expressive)            1  
3        4(offensive)            3  
4     3(entertaining)            2  


## 2. Model Structure

### Multimodal Fusion Design

In [14]:
import torch
import clip
from torch import nn

class MultimodalClassifier(nn.Module):
    def __init__(self, clip_model, num_classes=5):
        super().__init__()
        self.clip = clip_model
        # 冻结CLIP参数（可选）
        for param in self.clip.parameters():
            param.requires_grad = False
        # 分类头
        self.classifier = nn.Sequential(
            nn.Linear(512 * 2, 256),  # 图像+文本特征拼接
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, images, texts):
        # 提取多模态特征
        image_features = self.clip.encode_image(images)  # [batch, 512]
        text_features = self.clip.encode_text(texts)     # [batch, 512]
        # 拼接特征
        combined = torch.cat([image_features, text_features], dim=1)  # [batch, 1024]
        # 分类
        logits = self.classifier(combined)
        return logits

## 3. Dataset and Data Loader

### Custermized Dataset Class

In [15]:
import os
from torch.utils.data import Dataset, DataLoader

class MemeDataset(Dataset):
    def __init__(self, df, image_dir, preprocess):
        self.df = df
        self.image_dir = image_dir
        self.preprocess = preprocess

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # 加载图像
        image_path = os.path.join(self.image_dir, row["image_file"])
        image = self.preprocess(Image.open(image_path))
        # 处理文本
        text = clip.tokenize([f"This image has a {row['Overall Tone']}"])
        # 标签
        label = torch.tensor(row["label_index"], dtype=torch.long)
        return image, text.squeeze(0), label

In [31]:
# 修改后的MemeDataset类
class MemeDataset(Dataset):
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # 调用预定义的load_image
        image = load_image(os.path.join(self.image_dir, row["image_file"]))
        # 调用预定义的process_text
        text = process_text(row["Overall Tone"])
        label = torch.tensor(row["label_index"], dtype=torch.long)
        return image, text.squeeze(0), label  # 保持维度一致

### Load the data

In [17]:
dataset = MemeDataset(df, image_path_MetMeme, preprocess)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

## 4. Training and Evaluation

### Training configuration

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = MultimodalClassifier(model).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

### Training Loop

In [30]:
for epoch in range(10):
    for images, texts, labels in dataloader:
        images, texts, labels = images.to(device), texts.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images, texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

RuntimeError: mat1 and mat2 must have the same dtype, but got Half and Float

### Evaluation Index

In [ ]:
from sklearn.metrics import classification_report

model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for images, texts, labels in test_dataloader:
        images, texts, labels = images.to(device), texts.to(device), labels.to(device)
        outputs = model(images, texts)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=encoder.classes_))

## Prediction

In [ ]:
def predict(image_path, text, model, preprocess, encoder):
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    tokenized_text = clip.tokenize([f"This image has a {text}"]).to(device)
    with torch.no_grad():
        logits = model(image, tokenized_text)
    prob = torch.softmax(logits, dim=1)
    return {
        "class": encoder.inverse_transform([logits.argmax().item()])[0],
        "confidence": round(prob.max().item(), 4)
    }

# 示例
result = predict("image_001.jpg", "humorous tone", model, preprocess, encoder)
print(result)  # 输出: {"class": "entertaining", "confidence": 0.9231}